In [1]:
import json
import numpy as np
from math import isnan

In [2]:
def detect_turn_left(track_coordinates):
    if len(track_coordinates) < 3:
        return "Not enough data to determine"

    angles = []
    for i in range(1, len(track_coordinates) - 1):
        vector1 = track_coordinates[i - 1] - track_coordinates[i]
        vector2 = track_coordinates[i + 1] - track_coordinates[i]

        # Calculate the angle between the vectors
        angle = np.arccos(np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2)))

        if isnan(angle) == False:
            # Convert angle to degrees
            angle_deg = np.degrees(angle)
            if isnan(angle_deg) == False:
                angles.append(angle_deg)

    # Calculate the average angle
    avg_angle = np.mean(angles)


    # Determine if the car is turning or going straight
    if avg_angle >= 45 and avg_angle <= 135:
        return True
    else:
        return False


In [18]:
def detect_hard_brake(track_coordinates, time = 0.1):
    if len(track_coordinates) < 40:
        return False

    speeds = []
    for i in range(40, len(track_coordinates)):
        zeroIdx = i - 5
        brakeIdx = zeroIdx - 30
        highIdx = brakeIdx - 5
        distance_zero = np.linalg.norm(track_coordinates[i] - track_coordinates[zeroIdx])
        distance_high = np.linalg.norm(track_coordinates[highIdx] - track_coordinates[brakeIdx])

        if distance_zero < 30 and distance_high > 200:
            return True

    return False

In [4]:
def process(data, frame, track_num):
    res = []
    while frame < len(data):
        frame_info = data[frame]
        flag = False
        for track_info in frame_info:
            if track_info['track_id'] == track_num:
                flag = True
                break
        if flag == False:
            break
        centroid = [0,0]
        centroid[0] = (track_info['left'] + track_info['right'])/2
        centroid[1] = (track_info['top'] + track_info['bottom'])/2
        res.append(centroid)
        frame += 1
    return np.array(res)
        

In [5]:
def event_detect(data, start_frame, track_num, event):
    # get the numpy array of all the coordinate sequences
    sequence = process(data, start_frame, track_num)

    # run left detection
    res = False
    if event == "turn-left":
        res = detect_turn_left(sequence)
    elif event == "hard-brake":
        res = detect_hard_brake(sequence)
    return res

In [10]:
from tqdm import tqdm
def run_check(file_path, event = "turn-left"):
    seen_track = []
    event_occurance = 0
    with open(file_path, 'r') as file:
        data = json.load(file)
    for i in tqdm(range(len(data))):
        frame = data[i]
        if frame == None:
            continue
        for track in frame:
            if track['track_id'] not in seen_track:
                if event_detect(data, i, track['track_id'], event):
                    event_occurance += 1
                seen_track.append(track['track_id'])
    print(event_occurance)
    return event_occurance, len(data)
    

In [21]:
event = "turn-left"
prefix = "warsaw/"
for i in range(6):
    file_path = prefix+str(i)+"-baseline.json"
    occurance, frame_num = run_check(file_path, event)
    print("For", event, "event, file ",i, "has average event occurance per frame of ", occurance/frame_num)

  0%|                                                 | 0/31000 [00:00<?, ?it/s]/var/folders/39/fvsq4djn0lz6rfqd7738typc0000gq/T/ipykernel_69990/791936468.py:11: RuntimeWarning: invalid value encountered in scalar divide
  angle = np.arccos(np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2)))
/var/folders/39/fvsq4djn0lz6rfqd7738typc0000gq/T/ipykernel_69990/791936468.py:11: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2)))
100%|███████████████████████████████████| 31000/31000 [00:03<00:00, 8638.46it/s]


787
For turn-left event, file  0 has average event occurance per frame of  0.025387096774193547


100%|███████████████████████████████████| 31000/31000 [00:05<00:00, 6109.50it/s]


951
For turn-left event, file  1 has average event occurance per frame of  0.03067741935483871


100%|███████████████████████████████████| 31000/31000 [00:08<00:00, 3786.41it/s]


...

1814
For turn-left event, file  3 has average event occurance per frame of  0.058516129032258064


100%|███████████████████████████████████| 31000/31000 [00:15<00:00, 2052.93it/s]


2182
For turn-left event, file  4 has average event occurance per frame of  0.07038709677419355


100%|███████████████████████████████████| 25037/25037 [00:08<00:00, 2958.03it/s]

1619
For turn-left event, file  5 has average event occurance per frame of  0.0646642968406758
